# Notes


### Highway entry/exit
- Allgemein: <br>
    - Wenn anderes Auto neben ego ist, dann wird es eigetnlich nur schräg vor/hinter ego angezeigt und nicht parallel
    - Code refactoring
    - doc strings updaten
    - Kommentare anpassen
- dual exit/entry <br>
    - degree and col_distances anpassen, ggf. 2 unterschiedliche shapes für duals berücksichtigen
    - und runtern fahren / rauf fahren 
- single exit/entry und exit&entry
    - degree and col_distances anpassen
- auf exit abfahren / entry rauffahren:
    - mit exit noch mal checken

- Problem Kreuzungen:
    - Tankestelle und drum herum
    - double entry mit Ampel 
        - Weiter mit Situation, dass entry von links kommt
        - und es muss gefixt werden, dass ego_on_highway nicht erneuert wird, wenn ego auf highway_junction ist 

- Server
    - Fps fixen
    - UE Ordner überall pullen und Path anpassen




In [ ]:
# ToDo
 
# NEU
# -------------------------------------------------------------------------------------
 # 1) Simplify everything   
 # 2) config schreiben
 # 3) Doc strings (später)
 
# JP 
#--------------------------------------------------------------------------------------
 # 1) exits
 # 2) Die ekligen junctions: Tankstelle und Highway Ampel (JP)
        # Einfach 4 Spure ballern
        # simpel wie anderen 
 # -----------------------------------------------------------------------------------


# ICH
#------------------------------------------------------------------------------------
 # 1) exitst
 # 2) Stadt Scenario testen 

# Initialize

In [1]:
import pygame
import carla
import random
import time
import math
import collections
from collections import Counter
import pandas as pd
import itertools
from copy import deepcopy
import warnings
import numpy as np
import random


warnings.filterwarnings("ignore")
from utils import *

pygame 2.5.2 (SDL 2.28.3, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
distance_to_junc = 20
direction_angle = 20
radius = 100

actor_list = []

for actor in actor_list:
    actor.destroy()
actor_list = []

client = carla.Client("localhost", 2000)
world = client.get_world()

blueprint_library = world.get_blueprint_library()
world_map = world.get_map()
spawn_points = world.get_map().get_spawn_points()
print(world.get_actors())

[Actor(id=24, type=spectator), Actor(id=25, type=traffic.unknown), Actor(id=26, type=traffic.unknown), Actor(id=27, type=traffic.yield), Actor(id=28, type=traffic.yield), Actor(id=29, type=traffic.yield), Actor(id=30, type=traffic.unknown), Actor(id=31, type=traffic.unknown), Actor(id=32, type=traffic.yield), Actor(id=33, type=traffic.stop), Actor(id=34, type=traffic.stop), Actor(id=35, type=traffic.stop), Actor(id=36, type=traffic.stop), Actor(id=37, type=traffic.stop), Actor(id=38, type=traffic.stop), Actor(id=39, type=traffic.stop), Actor(id=40, type=traffic.stop), Actor(id=41, type=traffic.speed_limit.40), Actor(id=42, type=traffic.speed_limit.40), Actor(id=43, type=traffic.speed_limit.90), Actor(id=44, type=traffic.speed_limit.90), Actor(id=45, type=traffic.speed_limit.90), Actor(id=46, type=traffic.speed_limit.90), Actor(id=47, type=traffic.speed_limit.90), Actor(id=48, type=traffic.speed_limit.90), Actor(id=49, type=traffic.speed_limit.90), Actor(id=50, type=traffic.speed_limit.

In [4]:
manual_car = world.get_actor(147)
ego_vehicle = world.get_actor(147)
#second_vehicle = world.get_actor(153)
print(ego_vehicle.get_location())

Location(x=-356.329376, y=20.108377, z=0.034961)


# New Code

In [7]:
def check_ego_on_highway(ego_vehicle_location, road_lane_ids, world_map):
    waypoints = []
    ego_waypoint = world_map.get_waypoint(ego_vehicle_location)

    # TODO: nur kurz zum testen drin:
    if ego_waypoint.is_junction:
        junction = ego_waypoint.get_junction()
        if junction.id == 1368:
            return True
    ##########
    
    waypoints.append(ego_waypoint)
    if ego_waypoint.get_left_lane() is not None:
        waypoints.append(ego_waypoint.get_left_lane())
    if ego_waypoint.get_right_lane() is not None:
        waypoints.append(ego_waypoint.get_right_lane())
    for wp in waypoints:
        ego_vehilce_road_id = wp.road_id
        lanes = []
        for id in road_lane_ids:
            if str(ego_vehilce_road_id) == id.split("_")[0]:
                lanes.append(id.split("_")[1])
        lanes = [int(lane) for lane in lanes]
        if len(lanes) >= 6 or (
            sorted(lanes) == list(range(min(lanes), max(lanes) + 1)) and len(lanes) >= 3
        ):
            return True

    return False

In [8]:
def get_exit_or_entry(wps, exit_or_entry, junction):
    junction_wps = junction.get_waypoints(carla.LaneType().Driving)
    first = []
    second = []

    # special case for junction 1368: traffic light junction on highway
    if junction.id == 1368:
        entry_road_ids = [1450, 1412, 1421, 1432, 1442]
        for start_wp, end_wp in junction_wps:
            if start_wp.road_id in entry_road_ids:
                first.append(start_wp)
                second.append(end_wp)
        return (first, second)  # first = start_wp, second = end_wp

    if exit_or_entry == "exit":
        order = ["end", "start"]
    elif exit_or_entry == "entry":
        order = ["start", "end"]

    for group in wps:
        if len(group) != 4:
            for wp in group:
                if wp[1] == order[0]:
                    first.append(wp[0])

    for first_wp in first:
        for wp_pair in junction_wps:
            if wp_pair[0].id == first_wp.id:
                second.append(wp_pair[1])
                break
            elif wp_pair[1].id == first_wp.id:
                second.append(wp_pair[0])
                break

    if not first or not second:
        print("Error in get_exit()")
        return None
    if exit_or_entry == "exit":
        return (second, first)  # first = end_wp, second = start_wp
    elif exit_or_entry == "entry":
        return (first, second)  # first = start_wp, second = end_wp

In [9]:
def update_matrix(
    world_map,
    ego_vehicle,
    ego_location,
    highway_shape,
    wps,
    matrix,
    ego_waypoint,
    junction,
    cars_on_entryExit,
    direction_angle,
    ghost=False,
    on_entry = False,
):
    if highway_shape[1] == 3:
        matrix[list(matrix.keys())[5]] = [0, 0, 0, 0, 0, 0, 0, 0]

    matrix = list(matrix.items())
    exit_entry_found = False
    entry_wps, exit_wps = highway_shape[2], highway_shape[3]

    # TODO: put in a config file
    # set the angle threshold depending on curve type
    if highway_shape[0] == "entry_and_exit":
        degree = 30
    elif highway_shape[0] in ["single_entry", "single_exit"]:
        degree = 20
    elif highway_shape[0] in ["dual_entry"] or highway_shape[0] in ["dual_exit"]:
        degree = 18  # TODO: test and fine tuning of degree
    elif highway_shape[0] in [ "traffic_light_junction_entry_left", "traffic_light_junction_entry_right"]:
        degree = 30
    else:
        # just in case
        degree = 20

    # get right_lane_end_wp & right_lane_start_wp for dual entry & exit surrounding cars angle calculation
    if highway_shape[0] in ["dual_entry", "dual_exit"]:
        for wps_group in wps:
            if (len(wps_group) == 4) and (wps_group[0][1] == "end"):
                right_lane_end_wp = get_right_lane_wp(wps_group)
            elif (len(wps_group) == 4) and (wps_group[0][1] == "start"):
                right_lane_start_wp = get_right_lane_wp(wps_group)
    else:
        right_lane_end_wp = None
        right_lane_start_wp = None

    # First: Look behind ego
    exit_entry_found, col_entryExit, matrix = search_entry_or_exit(
        world_map,
        ego_location,
        "behind",
        exit_entry_found,
        matrix,
        highway_shape,
        wps,
        entry_wps,
        exit_wps,
        degree,
        junction.id,
        right_lane_end_wp,
        right_lane_start_wp,
    )
    # if not yet found, look in front
    if exit_entry_found == False:
        _, col_entryExit, matrix = search_entry_or_exit(
            world_map,
            ego_location,
            "before",
            exit_entry_found,
            matrix,
            highway_shape,
            wps,
            entry_wps,
            exit_wps,
            degree,
            junction.id,
            right_lane_end_wp,
            right_lane_start_wp,
        )

    # get road ids of: entry/exit, city road before and highway road after junction object
    entry_road_id = []
    exit_road_id = []
    entry_city_road = []
    exit_city_road = []
    entry_city_lane_id = []    
    exit_city_lane_id = []
    entry_highway_road = []
    exit_highway_road = []
    entry_highway_lane_id = []
    exit_highway_lane_id = []

    if entry_wps:
        for entry_wp in entry_wps[0]:
            entry_city_road.append(entry_wp.previous(3)[0].road_id)
            entry_city_lane_id.append(abs(entry_wp.previous(3)[0].lane_id))
            entry_road_id.append(entry_wp.road_id)
        for entry_wp in entry_wps[1]:
            entry_highway_road.append(entry_wp.next(3)[0].road_id)
            entry_highway_lane_id.append(abs(entry_wp.next(3)[0].lane_id))
    if exit_wps:
        for exit_wp in exit_wps[1]:
            exit_city_road.append(exit_wp.next(3)[0].road_id)
            exit_city_lane_id.append(abs(exit_wp.next(3)[0].road_id))
            exit_road_id.append(exit_wp.road_id)
        for exit_wp in exit_wps[0]:
            exit_highway_road.append(exit_wp.previous(3)[0].road_id)
            exit_highway_lane_id.append(abs(exit_wp.previous(3)[0].lane_id))


    # detect surrounding cars on exit/entry
    for car in cars_on_entryExit:
        col = calculate_position_in_matrix(
            ego_location,
            ego_vehicle,
            car,
            dict(matrix),
            world_map,
            ego_vehicle.get_velocity(),
            ghost,
        )
        if col is None:
            continue
        # matrix = list(matrix)
        if matrix:
            ego_already_in_matrix = False
            for key, row in matrix:
                if 1 in row:
                    ego_already_in_matrix = True

            if (ego_already_in_matrix and car.id == ego_vehicle.id):
                continue
            other_car_waypoint = world_map.get_waypoint(car.get_location())
            other_car_road_id = other_car_waypoint.road_id
            other_car_lane_id = other_car_waypoint.lane_id
            
            if other_car_road_id in entry_city_road + exit_city_road and not on_entry:
                continue
            distance_clostest_starting_waypoint = distance(get_clostest_starting_waypoint(junction.get_waypoints(carla.LaneType().Driving), car.get_location()).transform.location, car.get_location())
            if other_car_road_id in entry_city_road + exit_city_road:
                if distance_clostest_starting_waypoint > 30:
                    continue
                elif distance_clostest_starting_waypoint < 15:
                    row = 6
                else:
                    row = 7
                if highway_shape[0] in ["entry_and_exit"]:
                    col_entry = col_entryExit + 2
                    col_exit = col_entryExit
                elif highway_shape[0] in ["dual_entry"]:
                    entry_city_lane_id.sort()
                    # dual entry
                    if abs(other_car_lane_id) == entry_city_lane_id[0]:
                        col_entry = col_entryExit
                    elif abs(other_car_lane_id) == entry_city_lane_id[1]:
                        col_entry = col_entryExit + 1


                elif highway_shape[0] in ["dual_exit"]:
                    entry_city_lane_id.sort()
                    # dual exit
                    if abs(other_car_lane_id) == entry_city_lane_id[0]:
                        col_entry = col_entryExit +1
                    elif abs(other_car_lane_id) == entry_city_lane_id[1]:
                        col_entry = col_entryExit
                else:
                    col_entry = col_entryExit
                    col_exit = col_entryExit
            


                if entry_city_road == exit_city_road:
                    if is_junction_behind(other_car_waypoint, 40):
                        check_junction = get_junction_behind(other_car_waypoint, 40)
                        road_lane_ids = get_all_road_lane_ids(world_map)
                        if is_highway_junction(car, other_car_waypoint, check_junction, road_lane_ids, direction_angle):
                            on_exit_street = True
                        else:
                            on_exit_street = False
                else:
                    on_exit_street = False
                if (other_car_road_id in entry_city_road) and (col_entry < 8) and not on_exit_street:
                    insert_in_matrix(matrix, car, ego_vehicle, col_entry, row)
                elif (other_car_road_id in exit_city_road) and (col_exit < 8) and on_exit_street:
                    insert_in_matrix(matrix, car, ego_vehicle, col_exit, row)
                continue
                
            distance_junction_end = get_distance_junction_end(
                other_car_waypoint
            )
            distance_junction_start = get_distance_junction_start(
                other_car_waypoint
            )
            if other_car_road_id in entry_road_id + entry_highway_road:
                if highway_shape[0] == "dual_entry": 
                    yaw_difference = abs(
                        right_lane_end_wp.transform.rotation.yaw
                    ) - abs(
                        right_lane_end_wp.previous(distance_junction_end)[
                            0
                        ].transform.rotation.yaw
                    )
                else:
                    yaw_difference = abs(entry_wps[1][0].transform.rotation.yaw) - abs(
                        entry_wps[1][0]
                        .previous(distance_junction_end)[0]
                        .transform.rotation.yaw
                    )
            elif other_car_road_id in exit_road_id + exit_highway_road:
                if highway_shape[0] == "dual_exit":
                    yaw_difference = abs(
                        right_lane_start_wp.transform.rotation.yaw
                    ) - abs(
                        right_lane_start_wp.next(distance_junction_start)[
                            0
                        ].transform.rotation.yaw
                    )
                else:

                    yaw_difference = abs(exit_wps[0][0].transform.rotation.yaw) - abs(
                        exit_wps[0][0]
                        .next(distance_junction_start)[0]
                        .transform.rotation.yaw
                    )
             
            already_in_curve = abs(yaw_difference) > degree
            already_after_curve = abs(yaw_difference) > degree * 2
            if not on_entry:
                if already_after_curve:
                    row = 7
                elif already_in_curve:

                    row = 6
                else:
                    row = 5
            else:
                row = 5
                """
                distance_bounding_box = distance(junction.bounding_box.location, car.get_location())
                print(distance_bounding_box)
                if distance_bounding_box < 30:
                    row = 5
                if distance_bounding_box < 40:
                    row = 6
                else:
                    row = 7
                """
            if already_in_curve:
                if highway_shape[0] in ["entry_and_exit"]:
                    col_entry = col_entryExit + 2
                    col_exit = col_entryExit
                elif highway_shape[0] in ["dual_entry"]:
                    # dual entry
                    entry_road_id = [
                        x for _, x in sorted(zip(entry_highway_lane_id, entry_road_id))
                    ]
                    entry_highway_lane_id.sort()
                    if other_car_road_id == entry_road_id[0]:
                        col_entry = col_entryExit
                    elif other_car_road_id == entry_road_id[1]:
                        col_entry = col_entryExit + 1
                elif highway_shape[0] in ["dual_exit"]:
                    # dual exit
                    exit_road_id = [
                        x for _, x in sorted(zip(exit_highway_lane_id, exit_road_id))
                    ]
                    exit_highway_lane_id.sort()
                    if other_car_road_id == exit_road_id[0]:
                        col_exit = col_entryExit + 1
                    elif other_car_road_id == exit_road_id[1]:
                        col_exit = col_entryExit  # + 1
                else:
                    col_entry = col_entryExit
                    col_exit = col_entryExit
            else:
                if (highway_shape[1] == 3) and (col <= (col_entryExit + 1)):
                    col_entry = col_entryExit + 2
                    col_exit = col_entryExit
                else:
                    if highway_shape[0] in ["dual_entry"]:
                        # dual entry
                        entry_road_id = [
                            x
                            for _, x in sorted(
                                zip(entry_highway_lane_id, entry_road_id)
                            )
                        ]
                        entry_highway_lane_id.sort()
                        if other_car_road_id == entry_road_id[0]:
                            row = row - 1
                    elif highway_shape[0] in ["dual_exit"]:
                        # dual exit
                        exit_road_id = [
                            x
                            for _, x in sorted(zip(exit_highway_lane_id, exit_road_id))
                        ]
                        exit_highway_lane_id.sort()
                        if other_car_road_id == exit_road_id[0]:
                            row = row - 1
                    col_entry = col
                col_exit = col

            entry_streets = entry_road_id + entry_highway_road
            exit_streets = exit_road_id + exit_highway_road
            if (other_car_road_id in entry_streets) and (col_entry < 8):
                insert_in_matrix(matrix, car, ego_vehicle, col_entry, row)

            elif (other_car_road_id in exit_streets) and (col_exit < 8):
                insert_in_matrix(matrix, car, ego_vehicle, col_exit, row)

    return dict(matrix)

In [10]:
def search_entry_or_exit(
    world_map,
    ego_vehicle_location,
    direction,
    exit_entry_found,
    matrix,
    highway_shape,
    wps,
    entry_wps,
    exit_wps,
    degree,
    junction_id,
    right_lane_end_wp,
    right_lane_start_wp,
):
    ego_waypoint = world_map.get_waypoint(ego_vehicle_location)

    # for dual entry/exit we compare yaw values based on most right highway lane
    if highway_shape[0] in ["dual_exit", "dual_entry"]:
        initial_wp = ego_waypoint
        if direction == "before":
            while True:
                tmp_right_lane = initial_wp.get_right_lane()
                if (tmp_right_lane is None) or (
                    tmp_right_lane.lane_type != carla.LaneType().Driving
                ):
                    break
                initial_wp = tmp_right_lane
        # for looking behind, we consider left line first bc. otherwise wp.previous() go on entry
        elif direction == "behind":
            while True:
                tmp_left_lane = initial_wp.get_left_lane()
                if (tmp_left_lane is None) or (
                    tmp_left_lane.lane_type != carla.LaneType().Driving
                ):
                    break
                initial_wp = tmp_left_lane

    # for other entry/exit we compare yaw on entry/exit
    else:
        initial_wp = ego_waypoint

    col_entryExit = None
    # Look in front
    if direction == "before":
        z = 3  # column help in matrix
        col_distances = [6, 15, 35, 55, 75]
    # Look behind
    elif direction == "behind":
        z = (
            -1
        )  # column help in matrix; -1 because in list is first value only a threshold to see when junction too far behind
        if highway_shape[0] in [
            "entry_and_exit"
        ]:  # special because entry & exit are very close compared to normal column distance in matrix
            col_distances = [55, 35, 10, 6]
        elif highway_shape[0] in ["dual_exit"]:  # special because for dual exit
            col_distances = [55, 35, 20, 6]
        else:
            col_distances = [55, 35, 15, 6]
    for i, col_distance in enumerate(
        col_distances
    ):  # index 0 equals distance closest (10)
        if exit_entry_found:
            break
        for j in [
            -5,
            0,
            5,
        ]:  #  check 3 values in each column, bc. otherwise sometimes jumps are bigger than junction
            if exit_entry_found:
                break
            # Look in front
            if direction == "before":
                # if (col_distance == 4) and (j == 5):
                #     j = 0
                wp = initial_wp.next(col_distance + j)[0]
            # Look behind
            elif direction == "behind":
                # j = 0
                if highway_shape[0] == ["dual_entry", "dual_exit"]:
                    wp = initial_wp.previous(col_distance + j)[0]
                    while True:
                        tmp_right_lane = wp.get_right_lane()
                        if (tmp_right_lane is None) or (
                            tmp_right_lane.lane_type != carla.LaneType().Driving
                        ):
                            break
                        wp = tmp_right_lane
                else:
                    wp = initial_wp.previous(col_distance + j)[0]

            if (wp.is_junction) and (wp.get_junction().id == junction_id):
                # now we have to check if parallel or curve away, depending on exit / entry as first search
                if highway_shape[0] in [
                    "entry_and_exit",
                    "single_exit",
                ]:  # search for exit first
                    distance_junction_start = get_distance_junction_start(wp)
                    yaw_difference = abs(exit_wps[0][0].transform.rotation.yaw) - abs(
                        exit_wps[0][0]
                        .next(distance_junction_start)[0]
                        .transform.rotation.yaw
                    )
                    already_in_curve = abs(yaw_difference) > degree
                elif highway_shape[0] in ["single_entry"]:
                    distance_junction_end = get_distance_junction_end(wp)
                    yaw_difference = abs(entry_wps[1][0].transform.rotation.yaw) - abs(
                        entry_wps[1][0]
                        .previous(distance_junction_end)[0]
                        .transform.rotation.yaw
                    )
                    already_in_curve = abs(yaw_difference) < degree
                elif highway_shape[0] in ["dual_entry"]:
                    yaw_difference = abs(
                        right_lane_end_wp.transform.rotation.yaw
                    ) - abs(wp.transform.rotation.yaw)
                    already_in_curve = abs(yaw_difference) < degree
                elif highway_shape[0] in ["dual_exit"]:
                    yaw_difference = abs(
                        right_lane_end_wp.transform.rotation.yaw
                    ) - abs(wp.transform.rotation.yaw)
                    already_in_curve = abs(yaw_difference) > degree
                elif highway_shape[0] in ["traffic_light_junction_entry_right"]:
                    for entry_end_wp in entry_wps[1]:
                        if entry_end_wp.road_id == 1450:
                            distance_junction_end = get_distance_junction_end(wp)
                            yaw_difference = abs(
                                entry_end_wp.transform.rotation.yaw
                            ) - abs(
                                entry_end_wp
                                .previous(distance_junction_end)[0]
                                .transform.rotation.yaw
                            )
                            already_in_curve = abs(yaw_difference) < degree
                            break
                elif highway_shape[0] in ["traffic_light_junction_entry_left"]:
                    # 1 entry lane from left with 4 road_ids, starting at the same position and ending on 4 different highway lanes
                    for entry_end_wp in entry_wps[1]:
                        if entry_end_wp.road_id == 1421: # road_id 1421 is middle lane ending on 2nd left lane of highway
                            distance_junction_end = get_distance_junction_end(wp)
                            yaw_difference = abs(
                                entry_end_wp.transform.rotation.yaw
                            ) - abs(
                                entry_end_wp
                                .previous(distance_junction_end)[0]
                                .transform.rotation.yaw
                            )
                            already_in_curve = abs(yaw_difference) < degree
                            break
                else:
                    already_in_curve = False

                # if ego already too far away, dont show in matrix
                if (
                    (direction == "behind")
                    and already_in_curve
                    and (col_distance == 55)
                ):
                    exit_entry_found = True
                    break

                # if wp already in curve, update matrix
                if already_in_curve:
                    print(z+i)
                    matrix[6][1][z + i] = 0
                    matrix[7][1][z + i] = 0
                    col_entryExit = z + i
                    if (
                        z + i + 1 < 8
                    ):  # case: curve is 70m infront (last column in matrix)
                        if highway_shape[0] in ["entry_and_exit"]:
                            if highway_shape[1] == 3:
                                matrix[5][1][z + i + 1] = 3
                            # add entry, if closer than 70m
                            if z + i + 2 < 8:
                                matrix[6][1][z + i + 2] = 0
                                matrix[7][1][z + i + 2] = 0
                        elif highway_shape[0] in ["dual_entry"]:
                            matrix[6][1][z + i + 1] = 0
                            matrix[7][1][z + i + 1] = 0
                        elif highway_shape[0] in ["dual_exit"]:
                            matrix[6][1][z + i + 1] = 0
                            matrix[7][1][z + i + 1] = 0
                    if (z + i - 1 >= 0):
                        if highway_shape[0] in ["traffic_light_junction_entry_right"]:
                            matrix[6][1][z + i - 1] = 0
                            matrix[7][1][z + i - 1] = 0
                    exit_entry_found = True
                    break
    return exit_entry_found, col_entryExit, matrix


In [12]:
def get_highway_shape(wps_grouped, junction, ego_waypoint): #TODO new parameter ego_waypoint     
    # Check if entry or exit
    wps_grouped.sort(key=len)
    # special case for junction 1368: traffic light junction on highway
    if junction.id == 1368:
        if ego_waypoint.road_id == 41:
            highway_type = "traffic_light_junction_entry_left"
        elif ego_waypoint.road_id == 40:
            highway_type = "traffic_light_junction_entry_right"
        else:
            print("Error in get_highway_shape: Junction 1368 (traffic light junction on highway on Map04), ego vehicle not on road 40 or 41")
        straight_lanes = 4
    else:
        if len(wps_grouped[0]) == 2:
            if wps_grouped[0][0][1] != wps_grouped[0][1][1]:
                highway_type = "entry_and_exit"
            elif wps_grouped[0][0][1] == "start":
                highway_type = "dual_entry"
            elif wps_grouped[0][0][1] == "end":
                highway_type = "dual_exit"

        elif len(wps_grouped[0]) == 1:
            if wps_grouped[0][0][1] == "start":
                highway_type = "single_entry"
            else:
                highway_type = "single_exit"
        else:
            print("Junction object w/o shape")
            return ("normal_highway", 4, None, None)

        # Check number of lanes
        road_ids = [wp[0].road_id for wp in wps_grouped[1]]
        if len(set(road_ids)) == 1:
            straight_lanes = 4
        else:
            straight_lanes = 3
# TODO: STRAIGHT LANES UND ENTRY WPS /   EXIT WPS
    entry_wps = None
    exit_wps = None
    # get start & end waypoint of entry / exit as tuple
    if highway_type in ["entry_and_exit", "single_entry", "dual_entry", "traffic_light_junction_entry_left", "traffic_light_junction_entry_right"]:
        entry_wps = get_exit_or_entry(wps_grouped, "entry", junction)
    if highway_type in ["entry_and_exit", "single_exit", "dual_exit"]:
        exit_wps = get_exit_or_entry(wps_grouped, "exit", junction)

    return (highway_type, straight_lanes, entry_wps, exit_wps)

# Game Loop

In [13]:
same_junction = False
on_junction = False
junctions_detected = False
junction_old = None
wps_old = None
highway_shape_old = None
highway_shape = None
wrong_shape = None
junction_shape = None
matrix = None
junction_id = None
street_type = None 
road_lane_ids = get_all_road_lane_ids(world_map = world.get_map())
on_entry = False
junction_id_skip = None
df = initialize_dataframe()
#try:
while True:
    print("----------------------------------")
    ego_location = ego_vehicle.get_location()
    ego_waypoint = world_map.get_waypoint(ego_location)
    ego_on_highway = check_ego_on_highway(ego_location, road_lane_ids, world_map)
    if ego_on_highway:
        distance_to_junc = 80
    else:
        distance_to_junc = 20
    

    if junctions_detected and not ego_waypoint.is_junction:
        junctions_detected = False 
        same_junction = False
    elif junctions_detected:
        pass
    elif ego_waypoint.is_junction and on_junction == False:
        on_junction = True
    elif on_junction:
        junction_old = junction
        wps_old = wps
        highway_shape_old = highway_shape
        on_junction = False
        junctions_detected = True
    elif is_junction_ahead(ego_waypoint, distance_to_junc) and not on_junction: #and  != get_junction_ahead(ego_waypoint, distance_to_junc).id:
        junction = get_junction_ahead(ego_waypoint, distance_to_junc)
        junction_id = junction.id
        highway_junction = is_highway_junction(ego_vehicle, ego_waypoint, junction, road_lane_ids, direction_angle)


    # CASES
    # 1. On city junction
    if ego_waypoint.is_junction and not highway_junction and junction.id != 459:
        street_type = "On junction"
        print("On Junction")
        if junction_shape:
            matrix = detect_surrounding_cars_outside_junction(
                deepcopy(junction_shape),
                junction_roads_junction,
                lanes_all_junction,
                ego_vehicle,
                world,
                distance_to_junc,
                junction
            )
            matrix = detect_cars_inside_junction(
                matrix, ego_vehicle, junction, yaw, world
            )
            for key, value in matrix.items():
                print(key, value)
        
        else:
            print("Error in recognizing junction shape")
    
    # 2. City junction ahead
    elif is_junction_ahead(ego_waypoint, distance_to_junc) and not highway_junction and junction.id != 459:
        street_type = "Junction ahead"
        junction = get_junction_ahead(ego_waypoint, distance_to_junc)
        junction_shape, lanes_all_junction, junction_roads_junction, yaw, = getJunctionShape(
            ego_vehicle, ego_waypoint, junction, road_lane_ids, direction_angle
        )

        if junction_shape != "Error":
            print("Junction ahead")
            matrix = detect_ego_before_junction(
                deepcopy(junction_shape),
                junction_roads_junction,
                lanes_all_junction,
                ego_waypoint.lane_id,
                ego_waypoint,
                distance_to_junc,
            )
            matrix = detect_surrounding_cars_outside_junction(
                matrix,
                junction_roads_junction,
                lanes_all_junction,
                ego_vehicle,
                world,
                distance_to_junc,
                junction
            )
            matrix = detect_cars_inside_junction(
                matrix, ego_vehicle, junction, yaw, world
            )

            for key, value in dict(matrix).items():
                print(key, value)
        
        else:
            print("Error in recognizing junction shape")



    # 3. Highway entry/exit in front or behind
    elif (is_junction_ahead(ego_waypoint, 80) or (is_junction_behind(ego_waypoint, 40) and street_type == "Highway with entry/exit")) and not on_entry and highway_junction and ego_on_highway and junction_id_skip != junction.id:
        street_type = "Highway with entry/exit"
        print("Highway Junction detected")
        if not same_junction and is_junction_ahead(ego_waypoint, 80):
            same_junction = True
            # print("selbst erkennnen")
            junction = get_junction_ahead(ego_waypoint, 80)

            wps = junction.get_waypoints(carla.LaneType().Driving)
            closest_waypoint = get_clostest_starting_waypoint(wps, ego_location)
            wps = prepare_waypoints(closest_waypoint, wps, ego_waypoint)
            # # print grouped wps
            # for group in wps:
            #     print("######################### NEW GROUP #########################")
            #     for wp in group:
            #         print("-------------------")
            #         print(wp[0], wp[1])
            highway_shape = get_highway_shape(wps, junction, ego_waypoint) # TODO: New parameter ego_waypoint
            if not highway_shape is None:
                print(highway_shape[0], highway_shape[1])
        if (highway_shape is None) and (not junction_old is None):
            junction = junction_old
            wps = wps_old
            highway_shape = highway_shape_old
            wrong_shape = True

        matrix = create_city_matrix(ego_location, road_lane_ids, world_map)            
        if matrix:
            matrix, cars_on_entryExit = detect_surronding_cars(
                ego_location, ego_vehicle, matrix, road_lane_ids, world, radius, ego_on_highway, highway_shape
            )
            if not highway_shape is None and (highway_shape[0] != "normal_highway" or wrong_shape):
                matrix = update_matrix(world_map, ego_vehicle, ego_location, highway_shape, wps, matrix, ego_waypoint, junction, cars_on_entryExit, direction_angle) 
                if not junction_old is None and junction_old.id != junction.id:
                    matrix = update_matrix(world_map, ego_vehicle, ego_location, highway_shape_old, wps_old, matrix, ego_waypoint, junction_old, cars_on_entryExit, direction_angle) 
                
        for key, value in matrix.items():
            print(value)
        print("----------------------------------------------------------------------------------------------------------------------")


    # 4. Highway entry / exit
    elif is_junction_ahead(ego_waypoint, 20)  and highway_junction and street_type != "Highway with entry/exit":
        print("On highway entry")
        street_type = "On highway entry"
        on_entry = True
        junction = get_junction_ahead(ego_waypoint, 20)
        junction_id_skip = junction.id

        wps = junction.get_waypoints(carla.LaneType().Driving)
        closest_waypoint, middle_location = get_waypoint_on_highway_junction(ego_waypoint, ego_location, wps, world_map)
        wps = prepare_waypoints(closest_waypoint, wps, ego_waypoint)
        highway_shape = get_highway_shape(wps, junction, ego_waypoint) # TODO: New parameter ego_waypoint

        on_entry = True
        if (highway_shape is None) and (not junction_old is None):
            same_junction = False
            junction = junction_old
            wps = wps_old
            highway_shape = highway_shape_old


        matrix = create_city_matrix(middle_location, road_lane_ids, world_map, ghost=True)
        if matrix:
            matrix, cars_on_entryExit = detect_surronding_cars(
                middle_location, ego_vehicle, matrix, road_lane_ids, world, radius, True, highway_shape, ghost=True
            )

            if not highway_shape is None:
                matrix  = update_matrix(world_map, ego_vehicle, middle_location, highway_shape, wps, matrix, closest_waypoint, junction, cars_on_entryExit, direction_angle, True, on_entry, direction_angle) 
                # if not junction_old is None and junction_old.id != junction.id:
                #     matrix = update_matrix(world_map, ego_vehicle, middle_location, highway_shape_old, wps_old, matrix, closest_waypoint, junction_old, cars_on_entryExit, True, on_entry) 
                
        for key, value in matrix.items():
            print(value)
        print("----------")

    # 5. Normal Road
    else:
        # # test ###################
        # break
        on_junction = False
        junction_old = None
        on_entry = False
        print("normal road")
        same_junction = False
        if ego_on_highway:
            street_type = "On highway"
        else:
            street_type = "Non highway street"
        matrix = create_city_matrix(ego_location, road_lane_ids, world_map)
        if matrix:
            matrix, _ = detect_surronding_cars(
                ego_location, ego_vehicle, matrix, road_lane_ids, world, radius, ego_on_highway, highway_shape
            )
            for key, value in matrix.items():
                print(value)


    

    steering_angle = get_steering_angle(ego_vehicle)
    speed = get_speed(ego_vehicle)
    print("============================================================")
    if matrix:
        row_data = get_row(matrix)
        row_data["speed"] = speed
        row_data["steering_angle"] = steering_angle
        row_data["street_type"] = street_type
        # df = df.append(row_data, ignore_index=True)

    # clock.tick_busy_loop(60)
    time.sleep(0.3)
    world.tick()

    
"""
except Error:
    pass

finally:
    df.to_csv("x_hours.csv")
    vehicles_list = [ego_vehicle]
    print("\ndestroying %d vehicles" % len(vehicles_list))
    client.apply_batch([carla.command.DestroyActor(x) for x in vehicles_list])

    time.sleep(0.5)
"""

----------------------------------
Highway Junction detected
traffic_light_junction_entry_left 4
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
----------------------------------------------------------------------------------------------------------------------
----------------------------------
Highway Junction detected
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
----------------------------------------------------------------------------------------------------------------------
----------------------------------
Highway Junction detected
[3, 3, 3, 3, 3, 3, 3, 3]
[3, 3, 3, 3, 3, 3, 3, 3]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0,

KeyboardInterrupt: 

# Tests

In [5]:
# TEST: Print grouped wps in junction - current variable content
for group in wps:
    for wp in group:
        
        if wp[0]:
            print(wp[0].__str__(), wp[1])
            print(wp[0].road_id)
            print(wp[0].lane_id)
            #print("yaw:", wp[0].get_forward_vector())
    print("----------------------------------------------------------------------------------------------------------------------")

NameError: name 'wps' is not defined

In [38]:
# TEST: generate grouped wps in junction & highway shape
ego_location = ego_vehicle.get_location()
ego_waypoint = world_map.get_waypoint(ego_location)
ego_waypoint.is_junction
junction = ego_waypoint.get_junction()
junction = get_junction_ahead(ego_waypoint, 80)
wps = junction.get_waypoints(carla.LaneType().Driving)

closest_waypoint = get_clostest_starting_waypoint(wps, ego_location)
print(closest_waypoint)
#wps = prepare_waypoints(closest_waypoint, wps, ego_location)
#get_highway_shape(wps, junction, ego_waypoint)

Waypoint(Transform(Location(x=-357.734314, y=9.020967, z=0.003727), Rotation(pitch=-0.094554, yaw=179.631577, roll=0.000000)))


In [6]:
# TEST: get ego road/lane id loop
while True:
    time.sleep(0.3)
    ego_location = ego_vehicle.get_location()
    ego_waypoint = world_map.get_waypoint(ego_location)
    print("#####################")
    #print(is_junction_ahead(ego_waypoint, 80))
    print(ego_waypoint.road_id)
    #print(ego_waypoint.is_junction)

#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40
#####################
40


KeyboardInterrupt: 

In [27]:
# TEST: get ego information single moment
ego_location = ego_vehicle.get_location()
ego_waypoint = world_map.get_waypoint(ego_location)
# print(ego_waypoint.road_id)
print(is_junction_ahead(ego_waypoint, 80))
#is_junction_behind(ego_waypoint, 40)
# print(ego_waypoint.transform.location)

True


In [23]:
get_junction_ahead(ego_waypoint, 80).id

1368

In [ ]:
# TEST: get second vehicle information single moment
second_vehicle_location = second_vehicle.get_location()
second_vehicle_waypoint = world_map.get_waypoint(second_vehicle_location)
print(second_vehicle_waypoint.road_id)